### Generate a text leaning on previous n words. Nothing about ML, just math, ya know

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
data = pd.read_json("./arxivData.json")
data.sample(n=5)

,author,day,id,link,month,summary,tag,title,year
12519,"[{'name': 'Yi Zhou'}, {'name': 'Yingbin Liang'...",19,1802.06903v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,The success of deep learning has led to a risi...,"[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",Generalization Error Bounds with Probabilistic...,2018
7801,[{'name': 'Sean Billings'}],12,1803.04489v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",3,This paper explores the recently proposed Grap...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",Probabilistic and Regularized Graph Convolutio...,2018
33368,[{'name': 'Susan Khor'}],18,1107.3498v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",7,Slow self-avoiding adaptive walks by an infini...,"[{'term': 'cs.NE', 'scheme': 'http://arxiv.org...",What can we learn from slow self-avoiding adap...,2011
36574,"[{'name': 'Alexandru Baltag'}, {'name': 'Bryan...",27,1503.08141v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",3,The theory $\mathsf{CDL}$ of Conditional Doxas...,"[{'term': 'cs.LO', 'scheme': 'http://arxiv.org...",Revisable Justified Belief: Preliminary Report,2015
17409,"[{'name': 'Bernardt Duvenhage'}, {'name': 'Mfu...",1,1711.00247v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",11,Virtual assistants and text chatbots have rece...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Improved Text Language Identification for the ...,2017


In [5]:
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'].replace("\n", ' '), axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

In [6]:
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()

lines = [" ".join(tokenizer.tokenize(line)).lower() for line in lines]

In [7]:
sorted(lines, key=len)[0]

'differential contrastive divergence ; this paper has been retracted .'

In [8]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

In [26]:
from tqdm import tqdm
from collections import defaultdict, Counter
from typing import Tuple, LiteralString, List
from time import sleep


UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines, n) -> dict:
    res = defaultdict(Counter)

    for line in tqdm(lines, desc='Total'):
        corrected_line = (n - 1) * [UNK] + line.split() + [EOS]
        for word_ind in range(n - 1, len(corrected_line)):
            key = tuple(corrected_line[word_ind-n + 1:word_ind])
            res[key][corrected_line[word_ind]] += 1
        
    return res

In [27]:
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

Total: 100%|██████████| 100/100 [00:00<00:00, 16671.19it/s]


In [35]:
class NGramLanguageModel:    
    def __init__(self, lines, n):
        """ 
        Train a simple count-based language model: 
        compute probabilities P(w_t | prefix) given ngram counts
        
        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n
    
        counts = count_ngrams(lines, self.n)
        
        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)
        
        # populate self.probs with actual probabilities
        
        for key in counts.keys():
            for value in counts[key]:
                self.probs[key][value] = counts[key][value] / counts[key].total()
        
            
    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]
    
    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

In [36]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

Total: 100%|██████████| 100/100 [00:00<00:00, 24985.43it/s]


In [37]:
lm = NGramLanguageModel(lines, n=3)

Total: 100%|██████████| 41000/41000 [00:11<00:00, 3533.23it/s]


In [67]:
from random import choices

def get_next_token(lm, prefix, temperature=1.0):
    words = list(lm.get_possible_next_tokens(prefix).keys())

    if temperature == 0:
        probs = [lm.get_next_token_prob(prefix, word) for word in words]
        return words[probs.index(max(probs))]

    probs = [lm.get_next_token_prob(prefix, word) ** (1.0 / temperature) for word in words]

    word = choices(words, weights=probs)[0]

    return word

In [ ]:
from collections import Counter

test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

8643
Looks nice!


In [72]:
prefix = 'artificial' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

artificial neural network architectures ( including radial distortion is then refined by gaussian regression ; we address these challenges by developing a pe . the performance and increase revenue , especially for those classes ? is there a difference of samples . and adversarial ). experimental results demonstrate the ability of handling newly created problem specific characteristics . this camera image processing . recently , much of the marginal distribution function , and this is achieved using random sample consensus ( ransac ) approach . _EOS_


In [70]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

bridging the gap between the distributions , and the analysis of the user to the large - scale data sets . _EOS_
